In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import array
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [18]:
data = pd.read_csv('Dataset.csv')
data.head()

,text,pasal
0,"Pak A sering pegang pegang tangan, kadang pega...",1
1,lama lama dia tarik tanganku baru baring di pa...,1
2,pake jilbab syar'i dan gapernah sentuhan sama ...,0
3,"saya ditarik"" hampir masuk di ruangannya dosen...",1
4,saat mengajar dia mendekati teman saya dengan ...,1


In [19]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StemmerFactory()
stemmer_factory = factory.create_stemmer()
factory = StopWordRemoverFactory()
stopword_factory = factory.create_stop_word_remover()
def preprocessing(word):
    steaming = stemmer_factory.stem(word)
    finish = stopword_factory.remove(steaming)
    return finish

data.text = data.text.map(preprocessing)


In [28]:
x = data['text']
y = data['pasal']


In [29]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

In [30]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_token = '<OOV>'

In [31]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

training_padding = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padding = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

training_padding = np.array(training_padding)
training_labels = np.array(y_train)

testing_padding = np.array(testing_padding)
testing_labels = np.array(y_test)




In [51]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 16)           160000    
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               12544     
 nal)                                                            
                                                                 
 dense_36 (Dense)            (None, 1024)              66560     
                                                                 
 dense_37 (Dense)            (None, 1)                 1025      
                                                                 
Total params: 240,129
Trainable params: 240,129
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

num_epochs = 200
history = model.fit(training_padding, training_labels, epochs=num_epochs, batch_size=1,validation_data=(testing_padding, testing_labels), verbose=2)

Epoch 1/200


KeyboardInterrupt: 

In [15]:
sentence =["kemarin aku di kereta ada yang pegang-pegang tangan ku, pas udh lama malah dia makin parah malah pegang-pegang pantat ku"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(np.rint(model.predict(padded)))

[[1.]]


In [43]:
model.save('model.h5')